In [1]:
import requests
import lxml.html
import time
import sqlite3

In [54]:
# 列を追加。使用済みかどうかをチェック
# c.execute('alter table colums add column use interger')

In [5]:
conn = sqlite3.connect('sponavi_colum.db')

c = conn.cursor()  # カーソルを取得する。

# c.execute('DROP TABLE IF EXISTS colums')

# # columsテーブルを作成する。
# c.execute('''
#     CREATE TABLE colums (
#         URL text primary key,
#         Title text,
#         postdate text,
#         credit text,
#         author text,
#         article text
#      )

#  ''')

# 記事が重複するとエラーを起こすので、エラー処理。

for i,row in enumerate(clowl_sponavi_colum(c)):
    print(i)
    try:
        c.execute('INSERT INTO colums (URL, Title, postdate, credit, author, article) VALUES (:URL, :Title, :postdate, :credit, :author, :article)', row)
    except:
        pass

https://sports.yahoo.co.jp/column/detail/201805290001-spnavi?p=1
0


- 1page目 https://sports.yahoo.co.jp/column/list?category=/baseball&id=/npb
- 2page以降 https://sports.yahoo.co.jp/column/list?type=sportsnavi&id=%2Fnpb&category=%2Fbaseball&p=2

In [53]:
def clowl_sponavi_colum(cursor):
    for i in range(2):
        if i==0:
            # コンテンツのURLの一覧を取得
            htmls = parse_unit_page('https://sports.yahoo.co.jp/column/list?category=/baseball&id=/npb')
            for html in htmls:
                # すでに登録されている場合は詳細ページの処理はしない
                url = html.get('href')+'?p=1'
                cursor.execute('select count(*) from colums where URL="{}"'.format(url))
                if cursor.fetchall()[0][0]==0:
                    yield parse_unit_blog(html)
                else:
#                     print('ignore')
                    pass
        else:
            # コンテンツURLの一覧を取得
            htmls = parse_unit_page('https://sports.yahoo.co.jp/column/list?type=sportsnavi&id=%2Fnpb&category=%2Fbaseball&p='+str(i+1))
            for html in htmls:
                # すでに登録されている場合は詳細ページの処理はしない
                url = html.get('href')+'?p=1'
                cursor.execute('select count(*) from colums where URL="{}"'.format(url))
                if cursor.fetchall()[0][0]==0:
                    yield parse_unit_blog(html)
                else:
#                     print('ignore')
                    pass

In [3]:
# 1つの一覧ページを処理する。
def parse_unit_page(url):
    response = requests.get(url)
    parser = lxml.html.HTMLParser(encoding='utf-8')  # 文字コードの処理
    root = lxml.html.fromstring(response.content, parser=parser)
    return root.cssselect('p.articleTitle > a')  # コンテンツのURlを取得

In [4]:
# 一つのブログを処理する。
def parse_unit_blog(html):
    blog_contents = {}
    articles = []
    for i in range(10):
        time.sleep(2)
        response = requests.get(html.get('href')+'?p={}'.format(i+1))
#         response = requests.get(html+'?p={}'.format(i+1))
        parser = lxml.html.HTMLParser(encoding='utf-8')  # 文字コードの指定
        root = lxml.html.fromstring(response.content, parser=parser)
        
        
        # もし記事の最終校を過ぎてしまっていたら、for文から脱出して、
        # 著者情報を取得する。
        if root.cssselect('title')[0].text_content()=='エラーページ - スポーツナビ':
            if len(author)<=30:
                blog_contents['author'] = author
            else:
                blog_contents['author'] = ''
            
            blog_contents['article'] = '\n'.join(articles)
            break
        
        # 1ページ目の処理
        if i==0:
            print(response.url)
            blog_contents['URL'] = response.url
            blog_contents['Title'] = root.cssselect('.columnTitle')[0].text_content().strip()
            blog_contents['postdate'] = root.cssselect('.postDate')[0].text_content()
            blog_contents['credit'] = root.cssselect('.credit')[0].text_content()
        
        for article in root.cssselect('.columArticle > p'):
            articles.append(article.text_content())
        #最後のページには著者情報が乗っていることがある。
        else:
            author = article.text_content()
    
    return blog_contents

In [64]:
# データベースを閉じる
conn.close()